# 1084. Sales Analysis III

```sql
-- Write your PostgreSQL query statement below
SELECT p.product_id, p.product_name
FROM sales s
JOIN product p ON p.product_id = s.product_id
GROUP BY p.product_id, p.product_name
HAVING DATE('2019-03-31') >= MAX(s.sale_date) and MIN(s.sale_date) >= DATE('2019-01-01') 

In [ ]:
import pandas as pd
from pandas import to_datetime

def sales_analysis(product: pd.DataFrame, sales: pd.DataFrame) -> pd.DataFrame:
    
    # do a innter join
    new_df = product.merge(sales, on='product_id')

    # 2019-01-01 and 2019-03-31
    # .filter() works via the 'Split-Apply-Combine' strategy:
        # 1. SPLIT: new_df is cut into "mini-dataframes" (groups) based on 'product_id'.
        # 2. EVALUATE: Each group (x) is passed into the lambda condition.
        #    - The condition MUST return a single True or False for the entire group.
        # 3. COMBINE: 
        #    - If True: The entire group is kept in the final result.
        #    - If False: The entire group is discarded.

    
    # PERFORMANCE NOTE:
        # x['sale_date'].min() -> Pandas method (Vectorized)
        #   - Written in C; optimized for Series.
        #   - Processes millions of rows almost instantly by avoiding Python loops.
        #   - Naturally handles/ignores NaN values.
        #
        # min(x['sale_date']) -> Python built-in (Iterative)
        #   - Standard Python loop; pulls each value out of the Series one-by-one.
        #   - Significant overhead due to type conversion (Pandas -> Python object).
        #   - Much slower on large datasets and can fail on Nulls/NaNs.

    new_df = new_df.groupby(['product_id']).filter(lambda x: x['sale_date'].min() >= to_datetime("2019-01-01") and x['sale_date'].max() <= to_datetime("2019-03-31"))    

    return new_df[['product_id', 'product_name']].drop_duplicates()
    # then do a group by with a sort

In [1]:
import pandas as pd

data = [[1, 'S8', 1000], [2, 'G4', 800], [3, 'iPhone', 1400]]
product = pd.DataFrame(data, columns=['product_id', 'product_name', 'unit_price']).astype({'product_id':'Int64', 'product_name':'object', 'unit_price':'Int64'})
data = [[1, 1, 1, '2019-01-21', 2, 2000], [1, 2, 2, '2019-02-17', 1, 800], [2, 2, 3, '2019-06-02', 1, 800], [3, 3, 4, '2019-05-13', 2, 2800]]
sales = pd.DataFrame(data, columns=['seller_id', 'product_id', 'buyer_id', 'sale_date', 'quantity', 'price']).astype({'seller_id':'Int64', 'product_id':'Int64', 'buyer_id':'Int64', 'sale_date':'datetime64[ns]', 'quantity':'Int64', 'price':'Int64'})

In [21]:
# how to use min
array_1 = [1,2,34,5]
min(array_1)

1

In [21]:
# recap on using lambda

x = lambda a: a+10 
print(x(2))

x = lambda a: "Yes" if a==2 else "No"

print(x(2))
print(x(3))

12
Yes
No


In [62]:
# redo
import pandas as pd
from pandas import to_datetime
def sales_analysis(product: pd.DataFrame, sales: pd.DataFrame) -> pd.DataFrame:
    
    # inner join
    new_df = product.merge(sales, on='product_id', how='inner')

    # group by
    new_df = new_df.groupby(['product_id'])

    new_df = new_df.filter(lambda x: x['sale_date'].max()<=to_datetime('2019-03-31') and x['sale_date'].min()>=to_datetime('2019-01-01'))
    
    return new_df[['product_id','product_name']].drop_duplicates()

In [63]:
sales_analysis(product,sales)

,product_id,product_name
0,1,S8
